<a href="https://colab.research.google.com/github/gerald-wambui/RentIn/blob/main/devfest_riftvalley.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'
#/content/kaggle.json

In [ ]:
!kaggle datasets download -d alxmamaev/flowers-recognition

 96% 216M/225M [00:01<00:00, 118MB/s]
100% 225M/225M [00:01<00:00, 121MB/s]


In [ ]:
!unzip \*.zip

In [ ]:
import os
os.chdir('content/flowers')
print(os.listdir())#/content/flowers-recognition.zip

FileNotFoundError: ignored

In [ ]:
# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

#model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models
import tensorflow as tf

#preprocess.
from keras.preprocessing.image import ImageDataGenerator

#dl libraraies
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical
# specifically for cnn
from keras.layers import Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import random as rn
# specifically for manipulating zipped images and getting numpy arrays of pixel values of images.
import cv2
import numpy as np
import glob
import os

from sklearn.metrics import accuracy_score
from sys import getsizeof

In [ ]:
IMG_SIZE=150
FLOWER_DAISY_DIR='/content/flowers/daisy'
FLOWER_SUNFLOWER_DIR='/content/flowers/sunflower'
FLOWER_TULIP_DIR='/content/flowers/tulip'
FLOWER_DANDI_DIR='/content/flowers/dandelion'
FLOWER_ROSE_DIR='/content/flowers/rose'

In [ ]:
# Create a dictionary to store counts for each directory
image_counts = {}

# Loop through the directories and count images
for directory, dir_name in zip([FLOWER_DAISY_DIR, FLOWER_SUNFLOWER_DIR, FLOWER_TULIP_DIR, FLOWER_DANDI_DIR, FLOWER_ROSE_DIR], ['Daisy', 'Sunflower', 'Tulip', 'Dandelion', 'Rose']):
    image_paths = glob.glob(os.path.join(directory, '*.jpg'))
    image_count = len(image_paths)
    image_counts[dir_name] = image_count

# Print the image counts for each directory
for dir_name, count in image_counts.items():
    print(f"Directory '{dir_name}' contains {count} images.")

Directory 'Daisy' contains 764 images.
Directory 'Sunflower' contains 733 images.
Directory 'Tulip' contains 984 images.
Directory 'Dandelion' contains 1052 images.
Directory 'Rose' contains 784 images.


In [ ]:
# Initialize empty lists for data and labels
X = []
y = []

# Define a function to load and preprocess images for a specific category
def load_and_preprocess_category(category, directory):
    image_paths = glob.glob(os.path.join(directory, '*.jpg'))
    for image_path in image_paths:
        img = cv2.imread(image_path)
        if img is not None:
            img = cv2.resize(img, (224, 224))  # Resize the image
            img = img / 255.0  # Normalize pixel values
            X.append(img)
            y.append(category)

# Call the function for each flower category
load_and_preprocess_category('Daisy', FLOWER_DAISY_DIR)
load_and_preprocess_category('Sunflower', FLOWER_SUNFLOWER_DIR)
load_and_preprocess_category('Tulip', FLOWER_TULIP_DIR)
load_and_preprocess_category('Dandelion', FLOWER_DANDI_DIR)
load_and_preprocess_category('rose', FLOWER_DANDI_DIR)
# Encode labels using LabelEncoder
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Convert the lists to NumPy arrays
X = np.array(X)
Y = np.array(y_encoded)

In [ ]:
X

In [ ]:

Y

array([0, 0, 0, ..., 4, 4, 4])

In [ ]:
print(len(X))
print(len(Y))

4585
4585


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training (70%) and temporary data (30%)
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.30, random_state=42)

# Split the temporary data into validation (15%) and testing (15%)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.50, random_state=42)

In [ ]:
# Define the CNN model
model = keras.Sequential([
    # Convolutional layer 1
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),

    # Convolutional layer 2
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Convolutional layer 3
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Flatten the output for dense layers
    layers.Flatten(),

    # Dense layer 1
    layers.Dense(128, activation='relu'),

    # Output layer with softmax activation for classification
    layers.Dense(5, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()

In [ ]:
# Training & Evaluation

model.fit(X_train,Y_train, epochs=10 , batch_size=32 )

In [ ]:
score = model.evaluate (X_test,Y_test)
print("Test Score:",score[0])
print("Test Accuracy:",score[1])

In [ ]:
def get_file_size(file_path):
    size = os.path.getsize(file_path)
    return size

In [ ]:
def convert_bytes(size, unit=None):
    if unit == "KB":
        return print('File size: ' + str(round(size / 1024, 3)) + ' Kilobytes')
    elif unit == "MB":
        return print('File size: ' + str(round(size / (1024 * 1024), 3)) + ' Megabytes')
    else:
        return print('File size: ' + str(size) + ' bytes')

In [ ]:
KERAS_MODEL_NAME = "tf_model_flowers_rec.h5"
model.save(KERAS_MODEL_NAME)

In [ ]:
convert_bytes(get_file_size(KERAS_MODEL_NAME), "MB")

In [ ]:
keras_model_size = get_file_size(KERAS_MODEL_NAME)
TF_LITE_MODEL_FILE_NAME = "tf_lite_model.tflite"



In [ ]:
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = tf_lite_converter.convert()
tflite_model_name = "tf_lite_model.tflite"
open(tflite_model_name, "wb").write(tflite_model)

13225216

In [ ]:
convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "KB")

File size: 12915.25 Kilobytes


In [ ]:
tflite_file_size = get_file_size(TF_LITE_MODEL_FILE_NAME)

In [ ]:
interpreter = tf.lite.Interpreter(model_path = TF_LITE_MODEL_FILE_NAME)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

Input Shape: [  1 128 128   3]
Input Type: <class 'numpy.float32'>
Output Shape: [1 5]
Output Type: <class 'numpy.float32'>


In [ ]:
interpreter.resize_tensor_input(input_details[0]['index'], (688, 128, 128, 3))
interpreter.resize_tensor_input(output_details[0]['index'], (688, 5))
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

Input Shape: [688 128 128   3]
Input Type: <class 'numpy.float32'>
Output Shape: [688   5]
Output Type: <class 'numpy.float32'>


In [ ]:
X_test.dtype

dtype('float64')

In [ ]:
test_imgs_numpy = np.array(X_test, dtype=np.float32)
test_imgs_numpy.dtype

dtype('float32')

In [ ]:
interpreter.set_tensor(input_details[0]['index'], test_imgs_numpy)
interpreter.invoke()
tflite_model_predictions = interpreter.get_tensor(output_details[0]['index'])
print("Prediction results shape:", tflite_model_predictions.shape)
prediction_classes = np.argmax(tflite_model_predictions, axis=1)

Prediction results shape: (688, 5)


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score

# Assuming test_imgs_numpy is your input data and Y is the ground truth labels

# Resize the input and output tensor shapes in the interpreter
interpreter.resize_tensor_input(input_details[0]['index'], (688, 128, 128, 3))
interpreter.resize_tensor_input(output_details[0]['index'], (688, 5))
interpreter.allocate_tensors()

# Get the updated input and output details after resizing
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

# Set the input tensor with your test data
interpreter.set_tensor(input_details[0]['index'], test_imgs_numpy)

# Invoke the interpreter to make predictions
interpreter.invoke()

# Get the predictions from the output tensor
tflite_model_predictions = interpreter.get_tensor(output_details[0]['index'])
print("Prediction results shape:", tflite_model_predictions.shape)

# Assuming Y contains ground truth labels, calculate accuracy
prediction_classes = np.argmax(tflite_model_predictions, axis=1)
acc = accuracy_score(prediction_classes, Y)

# Print the accuracy
print("Accuracy:", acc)


Input Shape: [688 128 128   3]
Input Type: <class 'numpy.float32'>
Output Shape: [688   5]
Output Type: <class 'numpy.float32'>
Prediction results shape: (688, 5)


ValueError: ignored

In [ ]:
print(prediction_classes)
acc = accuracy_score(prediction_classes, Y)
#score = tflite_model.evaluate (X_test,Y_test)
#print("Test Score:",score[0])
#print("Test Accuracy:",score[1])

In [ ]:
tflite_file_size/keras_model_size

0.3329913754502171

In [ ]:
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tf_lite_converter.optimizations = [tf.lite.Optimize.DEFAULT]
tf_lite_converter.target_spec.supported_types = [tf.float16]
tflite_model = tf_lite_converter.convert()

In [ ]:
TF_LITE_MODEL_FLOAT_16_FILE_NAME = "tf_quantized_flowers.tflite"
tflite_model_name = TF_LITE_MODEL_FLOAT_16_FILE_NAME
open(tflite_model_name, "wb").write(tflite_model)

In [ ]:
convert_bytes(get_file_size(TF_LITE_MODEL_FLOAT_16_FILE_NAME), "KB")

In [ ]:
tflite_float_16_file_size = get_file_size(TF_LITE_MODEL_FLOAT_16_FILE_NAME)

In [ ]:
tflite_float_16_file_size/keras_model_size

In [ ]:
TF_LITE_QUANT_MODEL_NAME = "tf_lite_quant_model.tflite"

In [ ]:
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tf_lite_converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = tf_lite_converter.convert()

In [ ]:
tflite_model_name = "tf_lite_quant_model.tflite"
open(tflite_model_name, "wb").write(tflite_model)

In [ ]:
convert_bytes(get_file_size(TF_LITE_SIZE_QUANT_MODEL_FILE_NAME), "KB")

In [ ]:
tflite_float_quant_file_size = get_file_size(TF_LITE_SIZE_QUANT_MODEL_FILE_NAME)

In [ ]:
tflite_float_quant_file_size/keras_model_size

In [ ]:
tflite_float_quant_file_size/ tflite_float_16_file_size

In [ ]:
import tensorflow_model_optimization as tfmot

model = tf.keras.Sequential([
   ...
])
# Quantize the entire model.
quantized_model = tfmot.quantization.keras.quantize_model(model)

# Continue with training as usual.
quantized_model.compile(...)
quantized_model.fit(...)